# SetUp

In [1]:
# 📦 Main data packages
import numpy as np          # Numerical operations (arrays, stats, etc.)
import pandas as pd         # Data manipulation using DataFrames
import os

# 📊 Stats & time series
import statsmodels.formula.api as smf  # Easy linear modeling with formulas like y ~ x
from statsmodels.tsa.seasonal import seasonal_decompose  # Split time series into trend, seasonality, residuals
from scipy.ndimage import gaussian_filter  # Smooth data (useful for visual smoothing)

# 📆 Calendar utilities
from calendar import monthrange     # Get number of days in a given month
from calendar import month_name     # Get month names (e.g., January)

# 🎨 Visualization libraries
import matplotlib.pyplot as plt     # Core plotting library
import matplotlib.patches as patches  # Add custom shapes (e.g., rectangles) to plots
import seaborn as sns               # High-level interface for beautiful statistical plots

# 🔧 Seaborn styling
sns.set_style(
    style='darkgrid',               # Background grid with white axes
    rc={'axes.facecolor': 'white', 'grid.color': '.8'}  # Customize grid and background color
)

# 🎨 Custom colors
NF_ORANGE = '#ff5a36'               # Brand/custom color - bright orange
NF_BLUE = '#163251'                # Brand/custom color - deep blue
cmaps_hex = ['#193251','#FF5A36','#696969', '#7589A2','#FF5A36', '#DB6668']  # Custom palette

# 🔧 Set seaborn palette and store it in sns_c
sns.set_palette(palette=cmaps_hex)             # Apply custom color palette globally
sns_c = sns.color_palette(palette=cmaps_hex)   # Save palette as variable to use colors manually

# 🧠 Plot settings for Jupyter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()              # Avoid time series plotting issues

# 📐 Global plot config
plt.rcParams['figure.figsize'] = [12, 6]       # Set default plot size
plt.rcParams['figure.dpi'] = 100               # Set default resolution (100 is good quality)

# Import Data

In [2]:
data_path = '/Users/sayo/personal_projects/sales-ops-auto/data'
merged_tables_df = pd.read_csv(os.path.join(data_path, 'merged_tables.csv'))

In [3]:
merged_tables_df.head()

,sales_agent,account,deal_stage,engage_date,close_date,close_value,sector,year_established,revenue,employees,office_location,close_value_per_employee,close_value_per_revenue
0,Moses Frase,Cancity,Won,2016-10-20,2017-03-01,1054.0,retail,2001.0,718.62,2448.0,United States,0.430556,1.466700
1,Darcel Schlecht,Isdom,Won,2016-10-25,2017-03-11,4514.0,medical,2002.0,3178.24,4540.0,United States,0.994273,1.420283
2,Darcel Schlecht,Cancity,Won,2016-10-25,2017-03-07,50.0,retail,2001.0,718.62,2448.0,United States,0.020425,0.069578
3,Moses Frase,Codehow,Won,2016-10-25,2017-03-09,588.0,software,1998.0,2714.90,2641.0,United States,0.222643,0.216583
4,Zane Levy,Hatfan,Won,2016-10-25,2017-03-02,517.0,services,1982.0,792.46,1299.0,United States,0.397998,0.652399


In [ ]:
merged_tables_df.info()

In [5]:
data_forecast_df = merged_tables_df.copy()

# Convert 'date' column to datetime format
# Convert 'close_date' and 'engage_date' columns to datetime format
data_forecast_df[['close_date', 'engage_date']] = data_forecast_df[['close_date', 'engage_date']].apply(pd.to_datetime, format='%Y-%m-%d')

In [6]:
data_forecast_df.head()

,sales_agent,account,deal_stage,engage_date,close_date,close_value,sector,year_established,revenue,employees,office_location,close_value_per_employee,close_value_per_revenue
0,Moses Frase,Cancity,Won,2016-10-20,2017-03-01,1054.0,retail,2001.0,718.62,2448.0,United States,0.430556,1.466700
1,Darcel Schlecht,Isdom,Won,2016-10-25,2017-03-11,4514.0,medical,2002.0,3178.24,4540.0,United States,0.994273,1.420283
2,Darcel Schlecht,Cancity,Won,2016-10-25,2017-03-07,50.0,retail,2001.0,718.62,2448.0,United States,0.020425,0.069578
3,Moses Frase,Codehow,Won,2016-10-25,2017-03-09,588.0,software,1998.0,2714.90,2641.0,United States,0.222643,0.216583
4,Zane Levy,Hatfan,Won,2016-10-25,2017-03-02,517.0,services,1982.0,792.46,1299.0,United States,0.397998,0.652399


In [7]:
data_forecast_df = data_forecast_df.assign(
    close_date = lambda x: pd.to_datetime(x['close_date'], format='%Y-%m-%d'),
    engage_date = lambda x: pd.to_datetime(x['engage_date'], format='%Y-%m-%d'),
    
    close_year = lambda x: x['close_date'].dt.year,
    close_month = lambda x: x['close_date'].dt.month,
    close_day = lambda x: x['close_date'].dt.day,
    close_dayofyear = lambda x: x['close_date'].dt.dayofyear,
    
    engage_year = lambda x: x['engage_date'].dt.year,
    engage_month = lambda x: x['engage_date'].dt.month,
    engage_day = lambda x: x['engage_date'].dt.day,
    engage_dayofyear = lambda x: x['engage_date'].dt.dayofyear,
)

In [8]:
data_forecast_df.head()

,sales_agent,account,deal_stage,engage_date,close_date,close_value,sector,year_established,revenue,employees,...,close_value_per_employee,close_value_per_revenue,close_year,close_month,close_day,close_dayofyear,engage_year,engage_month,engage_day,engage_dayofyear
0,Moses Frase,Cancity,Won,2016-10-20,2017-03-01,1054.0,retail,2001.0,718.62,2448.0,...,0.430556,1.466700,2017.0,3.0,1.0,60.0,2016.0,10.0,20.0,294.0
1,Darcel Schlecht,Isdom,Won,2016-10-25,2017-03-11,4514.0,medical,2002.0,3178.24,4540.0,...,0.994273,1.420283,2017.0,3.0,11.0,70.0,2016.0,10.0,25.0,299.0
2,Darcel Schlecht,Cancity,Won,2016-10-25,2017-03-07,50.0,retail,2001.0,718.62,2448.0,...,0.020425,0.069578,2017.0,3.0,7.0,66.0,2016.0,10.0,25.0,299.0
3,Moses Frase,Codehow,Won,2016-10-25,2017-03-09,588.0,software,1998.0,2714.90,2641.0,...,0.222643,0.216583,2017.0,3.0,9.0,68.0,2016.0,10.0,25.0,299.0
4,Zane Levy,Hatfan,Won,2016-10-25,2017-03-02,517.0,services,1982.0,792.46,1299.0,...,0.397998,0.652399,2017.0,3.0,2.0,61.0,2016.0,10.0,25.0,299.0
